<a href="https://colab.research.google.com/github/thiagoribeiro00/AI-LegalDocAnalyzer/blob/main/AI_LegalDocAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install autogen -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00


In [3]:
import os
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

config_list =[
    {
        'model': 'gpt-3.5-turbo',
        'api_key': ''
    }
]

# ReAct prompt adapted for legal planning and analysis
ReAct_prompt = """
You are a legal expert tasked with helping users review and plan legal documents.
You can analyze clauses, detect potential issues, and suggest improvements for legal soundness.
Use the following format:

Question: the input question or request
Thought: you should always think about what to do
Action: the action to take (if any)
Action Input: the input to the action (e.g., search query)
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question or request

Begin!
Question: {input}
"""

# Define the ReAct prompt message
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

os.makedirs("coding", exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

assistant = AssistantAgent(
    name="LegalPlannerAssistant",
    system_message="You are a legal planner tasked with analyzing legal documents and planning legal actions. Use only the tools provided, and reply TERMINATE when done.",
    llm_config={"config_list": config_list, "cache_seed": None},
)

# Sample legal document to analyze
legal_document = """
This Agreement is made effective as of March 1st, 2024, by and between Omega Software Corp. ("Party A") and Horizon Data Solutions ("Party B").

1. Termination: Either party may terminate this Agreement with 60 days' notice. In the case of a breach of any material obligation by Party B, Party A can terminate with 7 days' notice. However, any termination initiated by Party A for convenience will require the full 60 days' notice.

2. Confidentiality: Both parties agree to maintain confidentiality regarding any proprietary information for the duration of the Agreement and 5 years following its termination. However, Party B may share Party A's proprietary information with its subsidiaries under "mutually agreed-upon circumstances."

3. Liability: Party A will not be liable for any indirect, incidental, or punitive damages. Party B shall indemnify Party A against all third-party claims, except in cases where Party A's gross negligence or willful misconduct caused the claim.

4. Intellectual Property: Any software developed as part of this Agreement will remain the intellectual property of Party A. Party B shall retain rights to any modifications or derivative works created as a result of the software, which Party B may use in unrelated projects.

5. Dispute Resolution: All disputes arising under this Agreement will be settled by arbitration in the State of California, except that Party A reserves the right to seek injunctive relief in federal court if it deems necessary.
"""

message='Analyze the following legal document for key clauses, potential issues, and suggest improvements. Provide a high-level summary of the document without step-by-step reasoning:'


# Initialize caching for the LLM responses
with Cache.disk(cache_seed=43) as cache:
    user_proxy.initiate_chat(
        assistant,
        message=react_prompt_message,
        question=f"Analyze the following legal document for potential issues, key clauses, and suggest improvements:\n\n{legal_document}",
        cache=cache,
    )

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


User (to LegalPlannerAssistant):


You are a legal expert tasked with helping users review and plan legal documents. 
You can analyze clauses, detect potential issues, and suggest improvements for legal soundness. 
Use the following format:

Question: the input question or request
Thought: you should always think about what to do
Action: the action to take (if any)
Action Input: the input to the action (e.g., search query)
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question or request

Begin!
Question: Analyze the following legal document for potential issues, key clauses, and suggest improvements:


This Agreement is made effective as of March 1st, 2024, by and between Omega Software Corp. ("Party A") and Horizon Data Solutions ("Party B").

1. Termination: Either party may terminate this Agreement with 60 days' notice. In the case of a breach of any materi